## The RNN Composer 

Importing the neccessary libraries and tools

In [1]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint

#### MIDI File Processing

This code block utilizes the Music21 library to parse MIDI files, extracting musical notes and chords. It compiles these elements into a sequential list, encoding single notes by their pitch and chords by joining the normal order of their notes with dots. The resulting list, 'notes,' serves as input data for subsequent steps in a process involving neural network training. 

In [2]:
# Initialize an empty list to store notes and chords
notes = []

# Iterate over all MIDI files in the "music_data" directory
for file in glob.glob("music_data/*.mid"):
    # Parse the MIDI file using Music21's converter to get a list of notes and chords in the file
    midi = converter.parse(file)
    parsed_notes = None
    
    try:
        # Attempt to partition the music by instrument
        s2 = instrument.partitionByInstrument(midi)
        parsed_notes = s2.parts[0].recurse() 
    except:
        # If partitioning by instrument fails, use flat.notes
        parsed_notes = midi.flat.notes
    
    # Iterate over each note or chord in the parsed notes
    for element in parsed_notes:
        # Check if the element is a single note
        if isinstance(element, note.Note):
            # Append the pitch of the note to the notes list
            notes.append(str(element.pitch))
        # Check if the element is a chord
        elif isinstance(element, chord.Chord):
            # Append the chord by joining the normal order of its notes with dots
            notes.append('.'.join(str(n) for n in element.normalOrder))

# Calculate the number of unique notes and chords in the list
num = len(set(notes))


### Data Preparation and Encoding 

This code block prepares data for a neural network by creating input sequences and corresponding outputs for music generation. It sets the input sequence length to 100 notes/chords, creates a numerical mapping for pitchnames, and iterates through the 'notes' list to generate training data. The input sequences are then reshaped, normalized, and the outputs are one-hot encoded to facilitate training the neural network for music composition.

In [21]:
# Set the length of each input sequence to 100 notes/chords
sequence_len = 100

# Create a sorted set of unique pitchnames from the 'notes' list
pitchnames = sorted(set(item for item in notes))

# Create a dictionary to map each pitchname to a numerical value
note_in_num = dict((note, number) for number, note in enumerate(pitchnames))

# Initialize empty lists to store input sequences and their respective outputs
net_input = []
net_output = []

# Iterate over the 'notes' list to create input sequences and outputs
for i in range(0, len(notes) - sequence_len, 1):
    seq_in = notes[i:i + sequence_len]
    seq_out = notes[i + sequence_len]
    # Convert the input sequence and output to numerical values using the mapping dictionary
    net_input.append([note_in_num[char] for char in seq_in])
    net_output.append([note_in_num[seq_out]])

# Calculate the total number of input patterns
num_patterns = len(net_input)

# Reshape the input data to fit the neural network input shape
norm_input = numpy.reshape(net_input, (num_patterns, sequence_len, 1))

# Normalize the input data by dividing it by the total number of unique pitchnames
norm_input = norm_input / float(num)

# One-hot encode the output data
net_output = to_categorical(net_output)


### Model Training 

This code block defines a neural network for music generation using Keras with three LSTM layers, Dropout layers for regularization, Dense layers for 'tighter' input and output connections, and an Activation layer. The model is compiled with categorical cross entropy loss and RMSprop optimizer. During training, model checkpoints are implemented to save weights after each epoch, allowing for the flexibility to stop training at any point without losing progress, and the final trained model is saved to a file named 'my_model.hdf5'.

NOTE: Only run this step to if you wish to train the model yourself on a powerful GPU, otherwise use the weights that I have already made. 

In [ ]:
# Define the neural network model architecture using Keras Sequential API
model = Sequential()

# Add the first LSTM layer with 512 nodes, input shape based on net_input dimensions, recurrent dropout, and returning sequences
model.add(LSTM(
    512,
    input_shape=(norm_input.shape[1], norm_input.shape[2]),
    recurrent_dropout=0.3,
    return_sequences=True
))

# Add the second LSTM layer with 512 nodes, recurrent dropout, and returning sequences
model.add(LSTM(
    512,
    return_sequences=True,
    recurrent_dropout=0.3
))

# Add the third LSTM layer with 512 nodes
model.add(LSTM(512))

# Add Batch Normalization to improve training stability
model.add(BatchNormalization())

# Add Dropout layer for regularization to prevent overfitting
model.add(Dropout(0.3))

# Add a Dense layer with 256 nodes
model.add(Dense(256))

# Apply Rectified Linear Unit (ReLU) activation function
model.add(Activation('relu'))

# Add Batch Normalization for improved convergence
model.add(BatchNormalization())

# Add Dropout layer for regularization
model.add(Dropout(0.3))

# Add the final Dense layer with nodes equal to the number of unique pitchnames
model.add(Dense(num))

# Apply softmax activation function for multi-class classification
model.add(Activation('softmax'))

# Compile the model using categorical cross entropy loss and RMSprop optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Define a ModelCheckpoint callback to save the model weights during training
checkpoint = ModelCheckpoint(
    filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

# Create a list of callbacks, including the ModelCheckpoint
callbacks_list = [checkpoint]

# Train the model using the prepared input sequences and outputs, with 100 epochs and a batch size of 128
model.fit(net_input, net_output, epochs=100, batch_size=128, callbacks=callbacks_list)

# Save the trained model to a file
model.save('my_model.hdf5')


### Pre-Trained Model Loading

In [22]:
# Define the neural network model architecture using Keras Sequential API
model = Sequential()

# Add the first LSTM layer with 512 nodes, input shape based on net_input dimensions, recurrent dropout, and returning sequences
model.add(LSTM(
    512,
    input_shape=(norm_input.shape[1], norm_input.shape[2]),
    recurrent_dropout=0.3,
    return_sequences=True
))
# Add the second LSTM layer with 512 nodes, recurrent dropout, and returning sequences
model.add(LSTM(
    512,
    return_sequences=True,
    recurrent_dropout=0.3
))

# Add the third LSTM layer with 512 nodes
model.add(LSTM(512))

# Add Batch Normalization to improve training stability
model.add(BatchNormalization())

# Add Dropout layer for regularization to prevent overfitting
model.add(Dropout(0.3))

# Add a Dense layer with 256 nodes
model.add(Dense(256))

# Apply Rectified Linear Unit (ReLU) activation function
model.add(Activation('relu'))

# Add Batch Normalization for improved convergence
model.add(BatchNormalization())

# Add Dropout layer for regularization
model.add(Dropout(0.3))

# Add the final Dense layer with nodes equal to the number of unique pitchnames
model.add(Dense(num))

# Apply softmax activation function for multi-class classification
model.add(Activation('softmax'))

# Compile the model using categorical cross entropy loss and RMSprop optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Load the weights to each node
model.load_weights("weights/weights-improvement-99-0.3726-bigger.hdf5")

### Generating Notes

This code block leverages a trained neural network to generate a sequence of 500 musical notes. It initiates the process by randomly selecting an index from the input data, ensuring varied results upon rerun. The code iteratively predicts and decodes notes using the trained model, updating the input pattern for each step in the generation process. The int_to_note dictionary is crucial for decoding numerical predictions back into categorical note values.

In [23]:
# Choose a random index as the starting point for note generation
start = numpy.random.randint(0, len(net_input)-1)

# Create a dictionary mapping numerical values to pitchnames for decoding the network output
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

# Initialize the input pattern with the randomly chosen starting point
pattern = net_input[start]
prediction_output = []

# Generate 500 notes using the trained model
for note_index in range(500):
    # Reshape the input pattern to fit the neural network input shape
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    # Normalize the input data
    prediction_input = prediction_input / float(num)

    # Make a prediction using the trained model
    prediction = model.predict(prediction_input, verbose=0)

    # Find the index of the highest predicted value
    index = numpy.argmax(prediction)
    # Decode the numerical value to a pitchname using the mapping dictionary
    result = int_to_note[index]
    # Append the decoded pitchname to the prediction output
    prediction_output.append(result)

    # Update the input pattern by adding the index and removing the first element
    pattern.append(index)
    pattern = pattern[1:len(pattern)]


### Generating the midi file

This code block takes the output from the neural network, which consists of encoded representations of notes and chords, and decodes them to create a sequence of Note and Chord objects. The offset is incremented for each iteration to avoid note stacking in the final composition. The resulting musical composition is then saved as a MIDI file named '59_epoch_composition.mid'.

In [24]:
# Initialize offset and an empty list to store output notes
offset = 0
output_notes = []

# Create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # Check if the pattern represents a chord
    if ('.' in pattern) or pattern.isdigit():
        # If it's a chord, split the string into an array of notes
        notes_in_chord = pattern.split('.')
        notes = []
        # Loop through the notes and create Note objects for each
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        # Create a Chord object with the notes and set its offset
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        # Append the Chord object to the output notes list
        output_notes.append(new_chord)
    # If the pattern represents a single note
    else:
        # Create a Note object using the pitch from the pattern
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        # Append the Note object to the output notes list
        output_notes.append(new_note)

    # Increase offset for each iteration to prevent note stacking
    offset += 0.5

# Create a stream of Note and Chord objects
midi_stream = stream.Stream(output_notes)

# Write the generated music to a MIDI file
midi_stream.write('midi', fp='59_epoch_composition.mid')


'59_epoch_composition.mid'